### Analyzing results from Phasor-Handler

In [116]:
import os
import numpy as np
import pandas as pd
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from collections import OrderedDict
from matplotlib import cm
import matplotlib.lines as mlines
import math
from pathlib import Path

# list all the folders in C:\Users\user\2p\mice_cortical\Group1_CX3CR1XAi203_7sbaseline_1sstim_50x50 and put them in trace_paths
trace_root = r"C:\Users\user\2p\DCN\20250924_DCN_PS_mice\Slide1.dir\100Hz_L_FOV1_Zoomed-1758686430-833.imgdir"
trace_paths = [trace_root]
# if os.path.isdir(trace_root):
#     trace_paths = sorted([
#         os.path.join(trace_root, d)
#         for d in os.listdir(trace_root)
#         if os.path.isdir(os.path.join(trace_root, d))
#     ])
# else:
#     print(f"Trace root not found: {trace_root}")

trace_paths


['C:\\Users\\user\\2p\\DCN\\20250924_DCN_PS_mice\\Slide1.dir\\100Hz_L_FOV1_Zoomed-1758686430-833.imgdir']

In [117]:
traces = []
exps = []

for i in trace_paths:
    trace_path = os.path.join(i, "trace.txt")
    exp_path = os.path.join(i, "experiment_summary.pkl")
    
    df = pd.read_csv(trace_path, sep="\t")
    df["Path"] = Path(i).stem
    with open(exp_path, 'rb') as f:
        exp_data = pickle.load(f)
    traces.append(df)
    exps.append(exp_data)

In [118]:
exps[0]["stimulation_ms"] = [10000, 20000, 30000, 40000]

In [119]:
timeframe_ms = 5000 # 10 seconds window

# Create a list of [start, end] ms windows for each stimulation
frame_ms = [
    [
        [stim_ms - timeframe_ms, stim_ms + timeframe_ms]
        for stim_ms in exp["stimulation_ms"]
    ]
    for exp in exps
]

# timestamps should be a 1-D numpy array per experiment (not a nested list)
timestamps = [np.array(exp["time_stamps"]) for exp in exps]

def get_frame_indices(window, timestamps_arr):
    # timestamps_arr is a 1-D numpy array of time stamps (ms)
    start_idx = int(np.argmin(np.abs(timestamps_arr - window[0])))
    end_idx = int(np.argmin(np.abs(timestamps_arr - window[1])))
    return [start_idx, end_idx]

# Create a list of [start_idx, end_idx] for each window per experiment
frame_idx = [[get_frame_indices(window, ts) for window in frames] for frames, ts in zip(frame_ms, timestamps)]

# Store frame_idx back into each experiment dict so downstream code can access it
for exp, idxs in zip(exps, frame_idx):
    exp['frame_idx'] = idxs

print(f"Wrote frame_idx for {len(exps)} experiments (first entry: {frame_idx[0] if frame_idx else None})")


Wrote frame_idx for 1 experiments (first entry: [[105, 316], [316, 526], [526, 736], [736, 947]])


In [120]:
all_stubs = ['Green_Mean_ROI', 'Red_Mean_ROI', 'Trace_ROI']
present_stubs = [s for s in all_stubs if any(col.startswith(s) for col in traces[0].columns)]

In [121]:
traces_list = []
for i, df in enumerate(traces):
    # assign dataset ID so you know which experiment it came from
    df = df.copy()
    df["Group"] = i  

    long_df = pd.wide_to_long(
        df,
        stubnames=["Green_Mean_ROI", "Red_Mean_ROI", "Trace_ROI"],
        i=["Frame", "Group", "Path"],
        j="ROI",
        sep="",
        suffix=r"\d+"
    ).reset_index()

    # simplify names
    rename_map = {
        "Green_Mean_ROI": "Green_Mean",
        "Red_Mean_ROI": "Red_Mean",
        "Trace_ROI": "Trace"
    }
    long_df = long_df.rename(columns={k: v for k, v in rename_map.items() if k in long_df.columns})

    traces_list.append(long_df)

# merge all into one big long dataframe
all_long = pd.concat(traces_list, ignore_index=True)


In [122]:
import functools
import operator

# Create framed ROIs using reduce to combine conditions
frame_conditions = [
    all_long["Frame"].between(frame_idx[0][x][0], frame_idx[0][x][1]) 
    for x in range(len(frame_idx[0]))
]
framed_rois = all_long[functools.reduce(operator.or_, frame_conditions)]

In [123]:
import numpy as np
import math

def baseline_for_group_roi(g):
    # unique sorted frames in this group (Group + ROI)
    frames = np.sort(g['Frame'].unique())
    n_base = max(1, math.ceil(0.20 * len(frames)))
    base_frames = set(frames[:n_base])
    # mean Green_Mean for this ROI restricted to baseline frames
    return g[g['Frame'].isin(base_frames)]['Green_Mean'].mean()

# Compute Fgo per (Group, ROI)
Fgo_roi = (framed_rois.groupby(['Group', 'ROI'], group_keys=False)
           .apply(baseline_for_group_roi)
           .rename('Fgo_roi')
          )

# Merge baseline back on both Group and ROI
framed_rois = framed_rois.merge(Fgo_roi.reset_index(), on=['Group', 'ROI'], how='left')

# Compute adjustedF safely
den = framed_rois['Red_Mean'].astype(float)
num = framed_rois['Green_Mean'].astype(float) - framed_rois['Fgo_roi'].astype(float)
framed_rois['adjustedF'] = np.where(np.isfinite(den) & (den != 0), num / den, np.nan)

C:\Users\user\AppData\Local\Temp\ipykernel_31324\3362692743.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Fgo_roi = (framed_rois.groupby(['Group', 'ROI'], group_keys=False)


In [124]:
# Instead of assigning all trials Stim = 1, assign based on which time window they fall into
def assign_stim_id(frame, frame_idx_list):
    """Assign stimulation ID based on which time window the frame falls into"""
    for stim_id, (start_idx, end_idx) in enumerate(frame_idx_list, 1):
        if start_idx <= frame <= end_idx:
            return stim_id
    return 0  # No stimulation (shouldn't happen since we filtered to these frames)

# Apply stimulation IDs based on frame indices
framed_rois["Stim"] = framed_rois["Frame"].apply(
    lambda x: assign_stim_id(x, frame_idx[0])
)

# Verify the assignment
print("Stimulation assignments:")
print(framed_rois.groupby("Stim")["Frame"].agg(["min", "max", "count"]))

Stimulation assignments:
      min  max  count
Stim                 
1     105  316   7844
2     317  526   7770
3     527  736   7770
4     737  947   7807


In [125]:
from matplotlib.backends.backend_pdf import PdfPages

pass_only = False
activation_fn = False
activation_threshold = 0.01
sig_threshold = 2.0


rois = sorted(framed_rois["ROI"].unique())

all_valid_f_values = np.concatenate([
    framed_rois["adjustedF"].values
    for _, trial_df in framed_rois.groupby(["ROI", "Group"])
    if not np.any(np.isnan(trial_df["adjustedF"].values))
])

# Find the absolute maximum and add a 10% buffer for visual padding
if all_valid_f_values.size > 0:
    global_max_y = np.max(all_valid_f_values) * 1.1
    if global_max_y < 0.05:
        global_max_y = 0.05  # Ensure a minimum height for visibility
else:
    global_max_y = 0.1 # A sensible default if no data is found

# You can also define a consistent minimum value
global_min_y = -0.05

output_file = os.path.join(trace_root, "report.pdf")


n_cols = 5
n_rows = int(np.ceil(len(rois) / n_cols))
analysis_type = "absolute" if activation_fn else "relative"

group_merge_map = {0: 0}
group_legend_labels = {
    0: ""
}

In [127]:
with PdfPages(output_file) as pdf:
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows), dpi=150)
    axes = axes.flatten()
    handles_by_label = OrderedDict()
    scatter_label = None

    n_trials_for_legend = None  # Will store the n_trials for legend

    for ax_idx, roi in enumerate(rois):
        ax = axes[ax_idx]
        all_trials = []
        trial_groups = []
        trial_stims = []
        roi_df = framed_rois[
            (framed_rois["ROI"] == roi)
        ]
        for (group_idx, stim_idx), trial_df in roi_df.groupby(["Group", "Stim"]):
            adjustedF = trial_df["adjustedF"].values
            if np.any(np.isnan(adjustedF)):
                continue
            all_trials.append(adjustedF)
            merged_group = group_merge_map.get(group_idx, group_idx)
            trial_groups.append(merged_group)
            trial_stims.append(stim_idx)
        if not all_trials:
            ax.axis('off')
            continue
        min_len = min(len(t) for t in all_trials)
        trials_array = np.stack([t[:min_len] for t in all_trials])
        trial_groups = np.array(trial_groups)
        stim_x = min_len // 2

        # Baseline and mean
        baseline = trials_array[:, :int(0.2 * min_len)]
        baseline_mean = float(np.mean(baseline))
        baseline_std = float(np.std(baseline, ddof=1))
        mean_trace = np.mean(trials_array, axis=0)

        # Save n_trials for legend (same for all plots)
        if n_trials_for_legend is None:
            n_trials_for_legend = trials_array.shape[0]

        # Build a stable mapping of merged_group -> color index for this ROI
        unique_groups = list(dict.fromkeys(trial_groups.tolist()))
        # If legend keys cover a superset, ensure their order is preserved
        legend_keys = list(group_legend_labels.keys())
        # Create a base palette large enough
        cmap = cm.get_cmap('tab10')

        def group_to_color(g):
            # Prefer to use legend_keys ordering if g appears there; otherwise use the order in unique_groups
            if g in legend_keys:
                idx = legend_keys.index(g)
            else:
                try:
                    idx = unique_groups.index(g)
                except ValueError:
                    idx = 0
            return cmap(idx % cmap.N)

        # Plot each trial with its mapped color
        for i, t in enumerate(trials_array):
            merged_group = int(trial_groups[i])
            group_label = group_legend_labels.get(merged_group, f"Group {merged_group}")
            color = group_to_color(merged_group)
            label = group_label if (trial_groups[:i] == merged_group).sum() == 0 else None
            ax.plot(t, color=color, alpha=0.5, linewidth=1, label=label)
        ax.plot(mean_trace, color='black', lw=3, label=f'Average (n = {n_trials_for_legend})')
        ax.axvline(x=stim_x, color='red', linestyle='-', alpha=0.7, label='Stimulation Onset')

        # Significance/Activation logic
        if activation_fn:
            activation_idx = np.where(mean_trace > (baseline_mean + activation_threshold))[0]
            if activation_idx.size > 0:
                scatter = ax.scatter(
                    activation_idx, mean_trace[activation_idx], s=20, color='blue', zorder=5,
                    label=f'> {activation_threshold} vs baseline'
                )
                scatter_label = f'> {activation_threshold} vs baseline'
            # If any trial in this ROI had NaN, show actual n in title
            n_actual = len(all_trials)
            n_total = len(roi_df.groupby(["Group", "Stim"]))
            if n_actual < n_total:
                title = f'ROI {roi} (n = {n_actual})'
            else:
                title = f'ROI {roi}'
        else:
            sig_idx = np.where(np.abs(mean_trace - baseline_mean) > (sig_threshold * baseline_std))[0]
            if sig_idx.size > 0:
                scatter = ax.scatter(
                    sig_idx, mean_trace[sig_idx], s=20, color='red', zorder=5,
                    label=f'> {sig_threshold}σ vs baseline'
                )
                scatter_label = f'> {sig_threshold}σ vs baseline'
            n_actual = len(all_trials)
            n_total = len(roi_df.groupby(["Group", "Stim"]))
            if n_actual < n_total:
                title = f'ROI {roi} (n = {n_actual})'
            else:
                title = f'ROI {roi}'

        ax.set_title(title, fontsize=10)
        # Set max value of all iterations as max y lim
        ax.set_ylim(-0.1, 0.5)
        ax.set_xlabel("Frame", fontsize=8)
        ax.tick_params(axis='both', labelsize=8)

        # Collect unique handles and labels from this subplot
        h, l = ax.get_legend_handles_labels()
        for handle, label in zip(h, l):
            if label and label not in handles_by_label:
                handles_by_label[label] = handle

    # Hide any unused axes
    for ax in axes[len(rois):]:
        ax.axis('off')

    # Ensure the scatter legend is always present, even if no points were found
    import matplotlib.lines as mlines
    if activation_fn:
        scatter_label = f'> {activation_threshold} vs baseline'
        scatter_handle = mlines.Line2D([], [], color='blue', marker='o', linestyle='None', markersize=6, label=scatter_label)
    else:
        scatter_label = f'> {sig_threshold}σ vs baseline'
        scatter_handle = mlines.Line2D([], [], color='red', marker='o', linestyle='None', markersize=6, label=scatter_label)
    if scatter_label not in handles_by_label:
        handles_by_label[scatter_label] = scatter_handle

    # Adjust layout for legend space (increase bottom margin for 6+ items)
    plt.tight_layout(rect=[0, 0.13, 1, 1]) 
    fig.legend(
        handles_by_label.values(), handles_by_label.keys(),
        loc='lower center', ncol=7, frameon=True, fontsize='x-large',
        bbox_to_anchor=(0.5, 0.1)
    )

    pdf.savefig(fig, bbox_inches='tight')
    plt.close(fig)

print("✅ PDF report has been generated.")

C:\Users\user\AppData\Local\Temp\ipykernel_31324\3505622441.py:48: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab10')
C:\Users\user\AppData\Local\Temp\ipykernel_31324\3505622441.py:48: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab10')
C:\Users\user\AppData\Local\Temp\ipykernel_31324\3505622441.py:48: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab10')
C:\Users\user\AppData\Local\Temp\ipykernel_

✅ PDF report has been generated.
